## Best practices for releases

This notebook is a walkthrough of the Cytoscape Release build process. Use the BUILD_PARENT_DIR variable to specify a directory to build cytoscape in.

#### NOTE: Make sure BUILD_PARENT_DIR is in `.gitignore`

1. Update versions
1. Update Core apps to use new API
1. Build Installers
1. Test on different OSs
1. Deploy subrepos to nexus
1. Post the installers to http://github.com/cytoscape/cytoscape/releases
1. Deploy updated core apps

After release:
1. Merge the branch used to build installers into master and tag the release
1. If master release, update the version in dev branch
1. Update Cytoscape manual via https://github.com/cytoscape/cytoscape-manual
1. Update www.github.com/cytoscape/cytoscape.github.com (downloads page, roadmap, etc.)
1. Update system requirements script at https://github.com/cytoscape/cytoscape-admin-scripts/tree/master/system-checker


In [1]:
from subprocess import Popen, PIPE
import os
import shutil

Set the notebook directory. Note that this is system dependent.

In [2]:
NOTEBOOK_DIR = '/home/cybuilder/cytoscape-admin-scripts'

print(NOTEBOOK_DIR)

/home/cybuilder/cytoscape-admin-scripts


Set the Java environment. This is Java 11 for Cytoscape 3.8.0 and above.

Note that this is system dependent, and may need to be changed to reflect the system's configuration.

In [3]:
%env JAVA_HOME=/usr/lib/jvm/jre-11-openjdk

env: JAVA_HOME=/usr/lib/jvm/jre-11-openjdk


Set the MAVEN_HOME environment variable, as well as point the path to the correct Maven binaries.

Note that this is system dependent and may need to be changed to reflect the system's configuration.

In [4]:
%env MAVEN_HOME=/opt/maven
%env PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin

env: MAVEN_HOME=/opt/maven
env: PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin


Prepare the target directory and clone the cytoscape git repo.

In [5]:
print('Changing to directory: ' + NOTEBOOK_DIR)
os.chdir(NOTEBOOK_DIR)

# Point to build location (the directory to clone parent cytoscape into)
BUILD_PARENT_DIR = os.path.join(os.getcwd(), '3.8.0_build')
if not os.path.exists(BUILD_PARENT_DIR):
    os.mkdir(BUILD_PARENT_DIR)
else:
    shutil.rmtree(BUILD_PARENT_DIR)
    os.mkdir(BUILD_PARENT_DIR)

os.chdir(BUILD_PARENT_DIR)
![[ -d cytoscape ]] || git clone https://github.com/cytoscape/cytoscape
CYTOSCAPE_ROOT_DIR = os.path.join(BUILD_PARENT_DIR, 'cytoscape')
CYTOSCAPE_DIR = os.path.join(CYTOSCAPE_ROOT_DIR, 'cytoscape')

def cd(directory=BUILD_PARENT_DIR, *subdirs):
    if subdirs:
        directory = os.path.join(directory, *subdirs)
    if os.getcwd() != directory:
        os.chdir(directory)

Changing to directory: /home/cybuilder/cytoscape-admin-scripts
Cloning into 'cytoscape'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 565 (delta 18), reused 20 (delta 7), pack-reused 529
Receiving objects: 100% (565/565), 766.37 KiB | 0 bytes/s, done.
Resolving deltas: 100% (271/271), done.


In [6]:
STARTING_BRANCH = 'develop'     # develop for major release, release/3.X.X for minor release
NEW_BRANCH = 'release/3.8.0'    # branch to create that will be deployed
OLD_VERSION = '3.8.0-SNAPSHOT'  # Version to replace in maven files
NEW_VERSION = '3.8.0'           # New version to set across maven
OLD_RELEASE_VERSION = '3.7.2'   # Release Version to replace on website/version checker

## 1. Pull the develop branch of Cytoscape

In [7]:
cd(CYTOSCAPE_ROOT_DIR)
![[ -d cytoscape ]] || ./cy.sh init
cd(CYTOSCAPE_DIR)
!./cy.sh run-all "git checkout {STARTING_BRANCH}"

Target directory = 
Cytoscape project will be cloned to: /home/cybuilder/cytoscape-admin-scripts/3.8.0_build/cytoscape
Cloning into 'cytoscape'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 565 (delta 18), reused 20 (delta 7), pack-reused 529
Receiving objects: 100% (565/565), 766.37 KiB | 0 bytes/s, done.
Resolving deltas: 100% (271/271), done.
Cloning: parent (URI = git@github.com:cytoscape/cytoscape-parent.git)
Cloning into 'parent'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 1640 (delta 40), reused 34 (delta 9), pack-reused 1569
Receiving objects: 100% (1640/1640), 256.35 KiB | 0 bytes/s, done.
Resolving deltas: 100% (625/625), done.
~/cytoscape-admin-scripts/3.8.0_build/cytoscape/cytoscape/parent ~/cytoscape-admin-scripts/3.8.0_build/cytoscape/cytoscape
Already on 'develop

## 1a. Reset

In [ ]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git clean -f -d'
!./cy.sh run-all 'git reset --hard'

## 2. Create new branch locally

In [ ]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git checkout -b {NEW_BRANCH}'
# !./cy.sh run-all 'git pull' # only necessary if branch is pushed

In [ ]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git status'

## 3. Update versions in pom files and all documents

In [ ]:
cd(CYTOSCAPE_DIR)
!mvn versions:set -DnewVersion={NEW_VERSION}

In [ ]:
# then update the versions in the parent directory
cd(CYTOSCAPE_DIR, 'parent')
!mvn versions:set -DnewVersion={NEW_VERSION}

## 3a. Commit (or reset) versions in pom files

In [ ]:
cd(CYTOSCAPE_DIR)
#!mvn version:revert
!mvn versions:commit

cd(CYTOSCAPE_DIR, 'parent')
#!mvn version:revert
!mvn versions:commit

## 3b. Update version in other files (poms, executables)

In [ ]:
%%bash -s "$OLD_VERSION" "$NEW_VERSION" "$CYTOSCAPE_DIR"

cd "$3"

echo "* Updating version numbers in pom files..."
find . -name pom.xml | xargs sed -i "" -e "s/$1/$2/g"
echo "Done"

echo "* Updating version numbers in executables..."
ls gui-distribution/assembly/src/main/bin/cytoscape.* | xargs sed -i "" -e "s/$1/$2/g"
echo "Done"

In [ ]:
# HelpTaskFactory uses hard coded versioning.
cd(CYTOSCAPE_DIR, 'impl', 'core-task-impl', 'src', 'main', 'java', 'org', 
   'cytoscape', 'task', 'internal', 'help')
!echo "* Updating Manual link"
!ls HelpTaskFactory.jav* | xargs sed -i "" -e "s/{OLD_RELEASE_VERSION}/{NEW_VERSION}/g"

## 3c. Maven clean and check for outstanding occurrences of old version

In [ ]:
!echo "* Cleaning old installers"
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
!mvn clean -q
cd(CYTOSCAPE_DIR, 'gui-distribution', 'splash-launcher')
!mvn clean -q
!echo "Done"

In [ ]:
cd(CYTOSCAPE_DIR)
ESCAPED = OLD_VERSION.replace('.', '\\.')
!echo "* Showing occurrences of {ESCAPED} in `pwd`:" && grep -ri '{ESCAPED}' .

In [ ]:
cd(CYTOSCAPE_DIR)
ESCAPED = OLD_RELEASE_VERSION.replace('.', '\\.')
!echo "* Showing occurrences of {ESCAPED} in `pwd`:" && grep -ri '{ESCAPED}' .

## 4. Build Cytoscape and ensure no errors
This may take a while. Expect to build subrepos first before building from the root directory

In [ ]:
cd(CYTOSCAPE_DIR)
with open('build_output.txt', 'w') as outf:
    process = Popen('mvn -fae clean install -U -Dmaven.test.skip=true'.split(' '), 
                stdout=outf,
                cwd=CYTOSCAPE_DIR)
    process.wait()

print("Showing ERROR lines in build...")
!cat build_output.txt | grep ERROR

## 5. Commit changes to new branch

In [ ]:
# if successful, commit the changes
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git add .'
!./cy.sh run-all 'git commit -m "Update version to {NEW_VERSION}"'

In [ ]:
cd(CYTOSCAPE_DIR)
!./cy.sh status

## 6. Build Cytoscape installers
This requires Install4J to be configured on your machine via the instructions at the Cytoscape README

In [ ]:
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
%env MAC_KEYSTORE_PASSWORD=
!mvn clean install -U

## 7. Copying Cytoscape installers to weekly download page
When completed installer executables can be found in cytoscape/cytoscape/gui-distribution/packaging/target/media and compressed builds for Linux and Windows can be found in cytoscape/cytoscape/gui-distribution/assembly/target

all of which should be copied to /var/www/html/cytoscape-builds/Cytoscape-3.8/3.8.0-Release-Candidate

In [ ]:
!mkdir -p /var/www/html/cytoscape-builds/Cytoscape-3.8.0/3.8.0-Release-Candidate-5
!rsync -av --progress /home/cybuilder/cytoscape-admin-scripts/3.8.0_build/cytoscape/cytoscape/gui-distribution/packaging/target/media/* /var/www/html/cytoscape-builds/Cytoscape-3.8.0/3.8.0-Release-Candidate-5
!rsync -av --progress /home/cybuilder/cytoscape-admin-scripts/3.8.0_build/cytoscape/cytoscape/gui-distribution/assembly/target/*.{gz,zip} /var/www/html/cytoscape-builds/Cytoscape-3.8.0/3.8.0-Release-Candidate-5


- Update core apps to rely on deployed releases, test
- Deploy and push them to app store
- Merge develop into master branch
- Tag and release master, publish release
- Change dev branch version

## 8. Deploy JARs to Nexus

- Need to have our repository properly configured in ~/.m2/settings.xml
- Cytoscape Project POM will fail. This is expected

In [ ]:
cd(CYTOSCAPE_DIR)
!mvn clean deploy

## 9. Notarizing on idekerlab-macmini

The install4j generated .dmg must be notarized to run on macOS 10.15 and above. The .dmg will be sent to idekerlab-macmini and submitted for notarization. An email will be sent to William Markuske when the notarization process is complete. As soon as the .dmg is notarized it will work and no further action is needed.

In [ ]:
!rsync -av --progress /var/www/html/cytoscape-builds/Cytoscape-3.8.0/3.8.0-Release-Candidate-5/*.dmg idekerlab@idekerlab-macmini.ucsd.edu:~/apps_to_notarize/
!ssh idekerlab@idekerlab-macmini.ucsd.edu '~/notarizedmg.sh ~/apps_to_notarize/*.dmg Release-Candidate.$(date +%Y%m%d)'    

You can check the status of the notarization by pasting in the RequestUUID value into the following command and running.

In [ ]:
!ssh idekerlab@idekerlab-macmini.ucsd.edu '~/notarizestatus.sh 0bfde93a-cd86-4e13-8898-ea3eae71a239'


#### Create a new release with Cytoscape installers: https://github.com/cytoscape/cytoscape/releases/new

## 10. Update Cytoscape.org, manual, and admin-scripts

Publish JavaDocs by building http://code.cytoscape.org/jenkins/job/cytoscape-3-javadoc/

Generate Release Notes from JIRA https://cytoscape.atlassian.net/projects/CYTOSCAPE

In [ ]:
# Update cytoscape.org

cd(BUILD_PARENT_DIR)
![[ -d cytoscape.github.com ]] || git clone http://github.com/cytoscape/cytoscape.github.com

![[ -d cytoscape-manual ]] || git clone http://github.com/cytoscape/cytoscape-manual

### Cytoscape.org changes

#### Roadmap
cytoscape.github.com/roadmap.html
- Remove section from Vision
- Move release dates 6-8 months forward
- Add section to Bottom of page listing features

#### JS Files:
- js/download.js
  - Copy JSON block to previous versions
- js/setup_page.js
  - Change release version
  
#### Welcome Letter
- Use template at cellar/users/bsettle/Documents/Cytoscape/Releases/
- Must update all sections

#### Release Notes
- releasenotes.html
  - Update welcome letter link
  - Update Latest Release Notes and Older Versions list
- release_notes_3_X_X.html
  - Copied from previous release notes
  - Edit HTML sections to add div's about new features
  - Add screenshots

In [ ]:
cd(BUILD_PARENT_DIR, 'cytoscape.github.com')
!git add .
!git commit -m "Updated for release {NEW_VERSION}"

### Manual
Refer to https://github.com/cytoscape/cytoscape-manual/ for manual changes and building

In [ ]:
#Change docs/conf.py
cd(BUILD_PARENT_DIR, 'cytoscape-manual', 'docs')
!ls conf.p* | xargs sed -i "" -e "s/{OLD_RELEASE_VERSION}/{NEW_VERSION}/g"
!git add .
!git commit -m "Updated for {NEW_VERSION} release"
!git push

- Update all sections with new features and screenshots
#### NOTE: ONLY TAG WHEN READY, Tagging posts the manual to ReadTheDocs
#### Only Major version updates are published. Not minor

Webhooks may be discontinued 1/31/2019. Keep an eye out for auto-building ReadTheDocs
To build it manually, you must have access to the ReadTheDocs page:
- https://readthedocs.org/projects/cytoscape-working-copy/

In [ ]:
cd(BUILD_PARENT_DIR, 'cytoscape-manual')
!git tag {NEW_VERSION}

### Update cytoscape-admin-scripts

In [ ]:
#system-checker files are in this repo
cd(BUILD_PARENT_DIR)
!ls system-checker/* | xargs sed -i "" -e "s/{OLD_RELEASE_VERSION}/{NEW_VERSION}/g"
!git add .
!git commit -m "Updated system checker"

### Publish the manual, website, and create a release on github
Check status before pushing!

In [ ]:
# Bump up the system checker scripts
cd(BUILD_PARENT_DIR)
!git status
#!git push

# Tag and publish Manual
cd(BUILD_PARENT_DIR, 'cytoscape-manual')
!git status
#!git push origin {NEW_VERSION}

# Update website
cd(BUILD_PARENT_DIR, 'cytoscape.github.com')
!git status
#!git push

### Update the file at /data/chianti/html/cytoscape-news/news.html

In [ ]:
!curl --silent http://chianti.ucsd.edu/cytoscape-news/news.html | grep "latestVersion"

### Announce the release on app-dev, cytoscape-announce, cytostaff

In [ ]:
updates = [
    'Java 11 Support',
    'Faster Renderering',
    'New User Interface Customization Options',
    'Updated apps, including CyBrowser, Analyzer, and CyNDEx-2',
    'Additional Graph Editing Tools'
]
update_list = '\n'.join([' - ' + update for update in updates])
welcome_letter = 'http://www.cytoscape.org/cy3_welcome_letter_v21.pdf'
release_notes = 'http://www.cytoscape.org/release_notes_3_8_0.html'
message = '''Cytoscape {version} is released!

=+=+ =+=+ =+=+ =+=+ =+=+ Hello Cytoscapers! =+=+ =+=+ =+=+ =+=+ =+=+
 
We're very pleased and excited to announce availability of Cytoscape {version}, effective immediately!

To get a copy, surf to our download page (http://cytoscape.org/download), and download and execute the installer for your workstation.

There are many new and exciting features in {version}, including:
{update_list}

Note that with {version}, Windows & Mac systems can run Java versions 8, 9, and 10 without conflicting with Cytoscape -- it installs and uses a private Java 11.

For Linux, you must be sure Java 11 is installed and is first on the PATH or is named via the JAVA_HOME environment variable.

If you find any issues with this release, please let us know by posting to cytoscape-helpdesk@googlegroups.com … we very much value your feedback!

Please check the {version} materials in our Welcome Letter ({welcome_letter}) and the Release Notes ({release_notes}).

Tell all your friends ...

We appreciate your checking this out, and we hope you find it as great to work with as we do!

=+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+'''
print(message.format(version='3.8.0', update_list=update_list, 
                     welcome_letter=welcome_letter, release_notes=release_notes))